In [ ]:
import sys, os
import itertools
from pathlib import Path
ROOT_DIR=Path(os.getcwd()).parent.parent
sys.path.append(str(ROOT_DIR))

from src.config import DB_DIR
from src.data.equity_data.etf.holdings import get_ark_etf_holdings, scrape_webpage
import src.data.equity_data.etf.btc_etf_holdings as BTC_ETF_Scraper

import streamlit as st
import plotly.express as px
import sqlite3
import datetime
import pandas as pd

In [ ]:
BTC_ETF_Scraper.get_holdings().set_index(['date','etf_ticker']).average_mv.unstack()

In [ ]:
fig = px.bar(
    BTC_ETF_Scraper.get_holdings().set_index(['date','etf_ticker'])\
        .btc_holdings.unstack().rename(pd.to_datetime)[['ARKB','IBIT','BRRR','BTCO','EZBC']]\
        .rename(columns={'ARKB': 'ARK 21Shares Bitcoin ETF', 
                         'IBIT': '贝莱德比特币ETF',
                         'BRRR': 'Valkyrie Bitcoin ETF',
                         'BTCO': 'Invesco Bitcoin ETF',
                         'EZBC': '富兰克林比特币ETF'}),
    barmode='stack',
    color_discrete_sequence=['#FFD300','#FF0800','#00A86B','#0080FF','#AF69EF'],
    template='plotly_dark'    
)
fig.update_layout(
    #title=f'比特币ETF持仓量（截止 {datetime.date.today().isoformat()}）',
    height=800, width=1200,
    title=f'比特币ETF持仓量（截止 2024-1-16）',
    xaxis_title='持仓日期',
    yaxis_title='比特币持有量',
    legend_title_text=None,
    # place legends on the top right corner
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0,
    ),
    # add annotations
    annotations=[
        dict(
            x=-.05,
            y=-0.1,
            showarrow=False,
            text="数据来源：ARK Invest, BlackRock, Valkyrie, Invesco, Franklin Templeton",
            xref="paper",
            yref="paper",
            font=dict(
                size=12,
                color="#ffffff"
            )
        )
    ]
)

